In [30]:
from calculate_colocations import get_pos_filtered_colloc_from_corpus_list, get_colloc_from_corpus_list,get_clean_lemm_list
import pandas as pd
from tqdm import tqdm
from collections import Counter

import json

In [31]:
texts_lenta = pd.read_csv("music_lenta.csv")

texts_lenta.iloc[0]['text']

'Расходы на проведение музыкального конкурса «Евровидение-2017» в Киеве оказались предельно высокими. Об этом в четверг, 11 мая, пишет украинское издание «Вести». «Украинцев возмутили запредельные гонорары артистов и ведущих шоу, то, что почти все тендеры проводились в сжатые сроки по сокращенной процедуре, а также явно лишние, неоправданные траты в стране, где идет война (речь идет о вооруженном конфликте в Донбассе — прим. «Ленты.ру», а большой части населения не хватает денег даже на оплату коммуналки», — говорится в заметке. По подсчетам газеты, на соревнование было потрачено 870 миллионов гривен (33 миллиона долларов). «Ровно столько закладывали в проект госбюджета Украины 2017 года на дотации государственными угольным шахтам (частичное покрытие разницы в себестоимости угля)», — пишет издание. По мнению авторов, пять миллионов гривен (190 тысяч долларов), потраченные на белое полотно, которым задрапировали «недострой рядом с местом проведения конкурса», «явно можно было бы потрати

In [32]:
clean_lemm_list = get_clean_lemm_list(list(texts_lenta['text']),lang = 'rus')

100%|██████████████████████████████████████████████████████████████████████████████| 5099/5099 [53:34<00:00,  1.59it/s]


In [33]:
clean_lemm_list[1]

'на'

In [34]:
unigramm_freq = Counter(clean_lemm_list)
with open ("music_unigr_freq.json","w", encoding = "utf-8") as f:
    json.dump(unigramm_freq, f, indent = 4, ensure_ascii = False)

In [35]:
bigramFreqTable, trigramFreqTable, quadgram_freq, filtered_bi, filtered_tri, bigramPMITable, trigramPMITable, quadragramPMITable, bigramChiTable, trigramChiTable =get_pos_filtered_colloc_from_corpus_list(list(texts_lenta['text']),"rus")


100%|██████████████████████████████████████████████████████████████████████████████| 5099/5099 [50:20<00:00,  1.69it/s]


In [36]:
def cean_pos_tags(df, ngramm_name):
    clean_words = []
    for i in tqdm(range(len(df))):
        posed_ngramm = df.iloc[i][ngramm_name]
        clean_ngramm = []
        for w in posed_ngramm:
            word = w.split("_")[0]
            clean_ngramm.append(word)
        clean_ngramm = tuple(clean_ngramm)
        clean_words.append(clean_ngramm)
    
    data = {"ngramm":clean_words,"freq":df['freq']}
    clean_df = pd.DataFrame(data) 
    return clean_df
bigramFreqTable_clean = cean_pos_tags(bigramFreqTable,'bigram')
trigramFreqTable_clean = cean_pos_tags(trigramFreqTable, "trigram")
quadgram_freq_clean = cean_pos_tags(quadgram_freq, "quadgramF")

100%|██████████████████████████████████████████████████████████████████████| 1044937/1044937 [06:46<00:00, 2567.61it/s]


In [37]:
filtered_bi_clean = cean_pos_tags(filtered_bi,'bigram')
filtered_tri_clean = cean_pos_tags(filtered_tri,'trigram')

100%|██████████████████████████████████████████████████████████████████████████| 96474/96474 [00:35<00:00, 2686.75it/s]


In [38]:
def get_freq_colloc_dict(ngrm_lis):
    freq_colloc_dict = {'2':{},'3':{},'4':{}}
    for ngramm_df in ngrm_lis:
        dct = ngramm_df.to_dict("split")
        #print(dct['data'][0][1],len(dct['data'][0][1]))
        words_len = len(dct['data'][0][1])
        for el in dct['data']:
            ngramm_raw = ''
            for el_i in el[1]:
                ngramm_raw += el_i + ' '
            ngramm_raw = ngramm_raw.strip()
            #print(ngramm_raw,words_len )
            freq_colloc_dict[str(words_len)][ngramm_raw] = el[1]
        
    return freq_colloc_dict
ngramms_list =[bigramFreqTable_clean, trigramFreqTable_clean, quadgram_freq_clean]
freq_colloc_dict = get_freq_colloc_dict(ngramms_list)

In [39]:
def get_united_numeric_colloc_base (numeric_calc_collocations_list, collocations_by_freq_dict):#все кроме фильтрованных по частям речи
    
    overall_colloc_json = {'2':{},'3':{},'4':{}}
    for colloc in numeric_calc_collocations_list:
        colloc_len = len(colloc)
        for key in list(colloc.keys()):
            if 'gram' in key:
                ngramm_name = key
            else:
                freq_name = key
        n_of_words = str(len(colloc.iloc[0][ngramm_name]))
        print(n_of_words)
        for index in tqdm(range(int(colloc_len))):
            collocation_element = colloc.iloc[index][ngramm_name]
            str_el = ''
            for el in collocation_element:
                str_el += el + ' '
            str_el = str_el.strip()
            if str_el in collocations_by_freq_dict[n_of_words]:
                overall_colloc_json[n_of_words][str_el] = collocations_by_freq_dict[n_of_words][str_el] 
            else:
                overall_colloc_json[n_of_words][str_el] = colloc.iloc[index][freq_name]
            """
            if str_el in overall_colloc_json[n_of_words]:
                overall_colloc_json[n_of_words][str_el] *= int(colloc.iloc[index][freq_name])
            else:
                overall_colloc_json[n_of_words][str_el] = int(colloc.iloc[index][freq_name])
            """
            #print(str_el)
        #print("========")
    return overall_colloc_json
#ngramms_list = [bigramFreqTable_clean, trigramFreqTable_clean, quadgram_freq_clean, bigramPMITable, trigramPMITable, quadragramPMITable]
ngramms_list = [bigramPMITable, trigramPMITable, quadragramPMITable, bigramChiTable, trigramChiTable,filtered_bi_clean,filtered_tri_clean]

log_collocations_vs_freq = get_united_numeric_colloc_base(ngramms_list,freq_colloc_dict)

2


100%|██████████████████████████████████████████████████████████████████████████| 13851/13851 [00:06<00:00, 2285.74it/s]


3


100%|██████████████████████████████████████████████████████████████████████████| 15890/15890 [00:05<00:00, 2719.97it/s]


4


...

100%|██████████████████████████████████████████████████████████████████████████| 15890/15890 [00:05<00:00, 2684.58it/s]


2


100%|████████████████████████████████████████████████████████████████████████| 103948/103948 [00:39<00:00, 2651.83it/s]


3


100%|██████████████████████████████████████████████████████████████████████████| 96474/96474 [00:37<00:00, 2586.46it/s]


In [40]:
with open ("music_smart_colloc_freq.json","w", encoding = "utf-8") as f:
    json.dump(log_collocations_vs_freq, f, indent = 4, ensure_ascii = False)